<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">TP : Requêtes SQL</h1>

## Principe de fonctionnement

Les requêtes SQL seront ici effectuées dans une base de données SQLite.

Cette base de données est contenue dans le fichier [`films.db`](Fichiers/films.db).

La structure de la base de données est décrite dans le document [`BDD_Films.pdf`](Fichiers/BDD_Films.pdf).

## Requêtes corrigées

* 10 noms de personnes

```sql
SELECT nom FROM personne LIMIT 10;
```

* Noms et dates de naissance des 10 premières personnes dont la date de naissance est connue (`IS NOT NULL`)  
Donner la liste dans l'ordre chronologique (du plus vieux au plus jeune).

```sql
SELECT nom, naissance FROM personne 
WHERE naissance IS NOT NULL 
ORDER BY naissance ASC LIMIT 10;
```

* Noms et dates de naissance des 10 premières personnes dont la date de naissance est connue (`IS NOT NULL`)
Donner la liste dans l'ordre chronologique inverse (du plus jeune au plus vieux).

*il y a des erreurs manifestement*

```sql
SELECT nom, naissance FROM personne  
WHERE naissance IS NOT NULL 
ORDER BY naissance DESC LIMIT 10;
```

* Liste des titres des films commençant par la lettre `M`, par ordre alphabétique 

```sql
SELECT titre FROM film 
WHERE titre LIKE 'M%';
```

* Différents noms des acteurs français (l'attribut `pays` doit valoir `France`)

Donner la liste dans l'ordre croissant des identifiants (attribut `id`), limité à 30 résultats

Attention, il ne faut pas lister une personne qui aurait été simplement réalisatrice, sans jamais jouer dans un film (dans ce cas ce n'est pas un acteur...)

```sql
SELECT DISTINCT nom FROM personne, joue 
WHERE personne.id = idActeur and pays='France'
ORDER BY id LIMIT 30;
```

* Nombre de réalisateurs par nationalité

Attention à bien comprendre l'intérêt de `DISTINCT` ici. 
Si on ne le met pas, un réalisateur ayant fait 2 films comptera deux fois.  
Au final, ce qu'on obtiendra, c'est le nombre de films, et non le nombre de réalisateurs...  
Notez comment on renomme une colonne, pour avoir un affichage plus propre.  
Notez aussi que la BDD contient quelques incohérences (U.S, U.S.A, United States... California ?...)

```sql
SELECT pays, COUNT(DISTINCT personne.id) AS nombre
FROM personne, film
WHERE personne.id = film.idRealisateur
GROUP BY pays;
```


* Titres des films (et le nombre d'acteurs) dont le nombre d'acteurs est supérieur à 40 par ordre décroissant du nombre d'acteurs

Notez comment on renomme la colonne `COUNT(*)` et comment on utilise son nom dans les clauses `HAVING` et `ORDER BY`.  
On peut aussi ne pas la renommer et écrire `HAVING COUNT(*) > 40`, mais c'est moins lisible.

```sql
SELECT titre, COUNT(*) as nbacteurs
FROM  joue, film
WHERE joue.idFilm = film.id
GROUP BY idFilm
HAVING nbacteurs > 40
ORDER BY nbacteurs DESC;
```

## Exercices

Dans la suite, vous devez écrire la requête qui fournit la réponse à la question rédigée. Chaque question indique bien la liste des colonnes qu'on souhaite voir. Il n'en faut ni plus, ni moins.

On utilise indifféremment dans la suite : ordre alphabétique, lexicographique, alphanumérique... Il s'agit dans tous les cas de l'ordre de SQL sur les chaînes de caractère (c'est l'ordre sur les codes ASCII des caractères)

Toutes les questions doivent être faites en une seule requête. Si on vous demande les films réalisés par Scorsese, par exemple, vous ne pouvez pas écrire une première requête pour relever le numéro `id` de Scorese, puis utiliser ce numéro dans une autre requête pour répondre à la question.

### 1. Requêtes simples
1.a. Liste des titres des films commençant par la lettre `M` et se terminant par la lettre `r` ou `s`

1.b Liste des titres des films sortis entre 2002 et 2004 (sur les 3 années)

1.c. Liste des noms et dates de naissance des personnes connues de la base dont le prénom commence par `Ro` et de nationalité Française (le pays vaut `France`), par ordre alphabétique.

### 2. Jointures

2.a. Liste des films sortis en 2006 en indiquant le titre, et le genre (en texte), par ordre alphabétique des titres.

2.b. Liste des drames sortis strictement avant 1970 en donnant le titre et l'année de sortie, par ordre chronologique (premier critère), puis alphabétique des titres (second critère de tri, pour les année de sorties égales)

2.c. Liste des genres de films réalisés par Martin Scorsese (sans que le même genre ne s'affiche deux fois).

2.d. Liste alphabétique des acteurs du film _Le Convoyeur_

Essayez de faire les jointures avec `WHERE` et `JOIN`.

2.e. Liste des films dont un des acteurs n'avait pas plus de 12 ans à la sortie du film.

Chaque ligne contiendra le titre du film, le nom de l'acteur, la date de sortie du film, la date de naissance de l'acteur et son âge à la sortie du film.  
Un film pourra sortir plusieurs fois si plusieurs acteurs sont concernés.  
Pour extraire une année numérique à partie d'une date de naissance, vous pouvez utiliser :

```sql
CAST(strftime("%Y", naissance) AS INTEGER)
```

La simple soustraction `annee - naissance` fonctionne aussi ici.

Il y a des entrées erronées dans la base... vous risquez d'avoir des âges négatifs (à cause de dates de naissance fausses), mais ce n'est pas à vous de régler ce problème :)

### 3. Agrégations
Souvent, dans ces questions, il est utile de d'abord produire une liste de réponses, et après avoir vérifié que c'était *a priori* correct, d'ajouter les opérateurs d'agrégation dans la requête.

3.a. Table contenant seulement l'année de sortie du plus ancien film, et l'année de sortie du film le plus récent.

3.b. Moyenne du nombre de spectateurs par film.

3.c. Table contenant, pour chaque genre, le nombre de films qui relèvent de ce genre, par ordre décroissant du nombre de films

3.d. Liste des 10 films comptant le plus d'acteurs, par ordre décroissant du nombre d'acteur, puis (2e critère) ordre alphabétique des titres

Attention, il pourrait y avoir des films différents avec le même titre !

3.e. Liste des films ayant le salaire moyen d'acteurs supérieur 2 millions, par ordre décroissant du salaire moyen.

Note : Le champ salaire a été rempli aléatoirement (unité €, dollars ou millions de dollars, c'est selon...).


3.f.  Recette totale générée par des films de réalisateurs américains (vous utiliserez pays = 'USA')  sortis en 2010, en prenant un prix de l'entrée à 8 €

### Sous-requêtes

**Liste des films (avec leur année de sortie) sortis au pire 2 ans après le film le plus ancien de la base.**

Vous n'avez pas le droit de chercher cette date, puis de la mettre en dur dans la requête, il faut que votre requête fonctionne sans modification *quelles que soient les données présentes dans la base*.

## Sources :
* [SQLite](https://www.sqlite.org/index.html)
* [DB Browser for SQLite](https://sqlitebrowser.org/)
* Base de données des films : [`films.db`](Fichiers/films.db) et sa structure est décrite dans le document [`BDD_Films.pdf`](Fichiers/BDD_Films.pdf)